### Hidden Markov Model (HMM)

Assume for simplicity, that my wife has 4 just states:
- happy
- tired 
- angry
- normal

She cannot say directly in which state she is right now (hidden states), and we can just guess using external observable information. Let's say, she can do one of the following actions: 
- cook
- slide Instagram
- study
- and do shopping

The goal is to understand her real (hidden) state looking at what she is doing and what she has done before this current moment. In other words, there is a sequence of observable states/actions which might be used to retrieve sequence of hidden states.

We can use **Hidden Markov Model** for this purpose.

In [1]:
import numpy as np
import pandas as pd
from nltk.tag import hmm
from sklearn.metrics import classification_report
from IPython.display import display

Let's define **true** parameters of HMM

In [2]:
# Hidden states
states = ["happy", "tired", "angry", "normal"]

# Observed states
symbols = ["cooking", "instagraming", "studying", "shopping"]

# Transition probabilities matrix
A_true = np.array([[0.5, 0.1, 0.1, 0.3],
                   [0.2, 0.4, 0.1, 0.3],
                   [0.1, 0.1, 0.6, 0.2],
                   [0.3, 0.2, 0.2, 0.3]], dtype=np.float64)

# Emission probabilities matrix
B_true = np.array([[0.6, 0.1, 0.1, 0.2],
                   [0.1, 0.7, 0.1, 0.1],
                   [0.2, 0.1, 0.1, 0.6],
                   [0.3, 0.3, 0.3, 0.1]], dtype=np.float64)

# Probabilities of initial states (prior)
pi_true = np.array([0.6, 0.2, 0.1, 0.1], dtype=np.float64)

print("Hidden states transition probabilities matrix:")
display(pd.DataFrame(data=A_true, index=states, columns=states))

print("\n\nObserved states emission probabilities matrix:")
display(pd.DataFrame(data=B_true, index=states, columns=symbols))

Hidden states transition probabilities matrix:


,happy,tired,angry,normal
happy,0.5,0.1,0.1,0.3
tired,0.2,0.4,0.1,0.3
angry,0.1,0.1,0.6,0.2
normal,0.3,0.2,0.2,0.3




Observed states emission probabilities matrix:


,cooking,instagraming,studying,shopping
happy,0.6,0.1,0.1,0.2
tired,0.1,0.7,0.1,0.1
angry,0.2,0.1,0.1,0.6
normal,0.3,0.3,0.3,0.1


Initialize HMM model with **true** parameters

In [3]:
model_true = hmm._create_hmm_tagger(states, symbols, A_true, B_true, pi_true)

Generate sequence of pairs (observable state, hidden state) using HMM.<br>
We have been living together with my wife for 5 years, so let's say we have 5 * 365 days sequence length.

In [5]:
seq_len = 5 * 365
seq = model_true.random_sample(rng=np.random, length=seq_len)

print("First 7 days:")
seq[:7]

First 7 days:


[('cooking', 'happy'),
 ('cooking', 'normal'),
 ('instagraming', 'tired'),
 ('instagraming', 'normal'),
 ('shopping', 'tired'),
 ('studying', 'normal'),
 ('cooking', 'happy')]

Let's pretend we do not know real (true) parameters of HMM.

Then we can **estimate** them using:
- supervised algorithm (MLE)
- unsupervised algorithm (Baum-Welch)
- both of them (i.e. Baum-Welch with initialized probabilities obtained from MLE estimation)

In [6]:
def get_probabilities_matrix(model, kind):
    """Returns probabilities in matrix form from fitted NLTK HMM Tagger"""
    source = states
    if kind == "transition":
        destination = model._states
        values = model._transitions
    elif kind == "emission":
        destination = model._symbols
        values = model._outputs
    else:
        raise ValueError("Invalid parameter 'kind', must be 'transition' or 'emission'")
    
    iters = (values[sj].prob(si) for sj in source for si in destination)
    probas = np.fromiter(iters, dtype=np.float64)
    N, M = len(source), len(destination)
    return probas.reshape((N, M))

Supervised estimation (MLE):

In [7]:
trainer = hmm.HiddenMarkovModelTrainer(states=states, symbols=symbols)
model_mle = trainer.train_supervised([seq])

A_mle = get_probabilities_matrix(model_mle, kind="transition")
B_mle = get_probabilities_matrix(model_mle, kind="emission")

print("Transition probabilities matrix [TRUE]:")
display(pd.DataFrame(data=A_true, index=states, columns=states))

print("Transition probabilities matrix [ESTIMATED]:")
display(pd.DataFrame(data=A_mle.round(2), index=states, columns=states))

Transition probabilities matrix [TRUE]:


,happy,tired,angry,normal
happy,0.5,0.1,0.1,0.3
tired,0.2,0.4,0.1,0.3
angry,0.1,0.1,0.6,0.2
normal,0.3,0.2,0.2,0.3


Transition probabilities matrix [ESTIMATED]:


,happy,tired,angry,normal
happy,0.48,0.10,0.10,0.32
tired,0.14,0.40,0.08,0.37
angry,0.11,0.10,0.62,0.18
normal,0.31,0.22,0.22,0.25


The estimation with supervised MLE approach is pretty good.

In [8]:
print("Supervised estimation...")
print("Average abs error, transition matrix: {:.3f}" \
          .format(np.abs(A_true - A_mle).mean()))
print("Average abs error, emission matrix: {:.3f}" \
          .format(np.abs(B_true - B_mle).mean()))

Supervised estimation...
Average abs error, transition matrix: 0.022
Average abs error, emission matrix: 0.013


... much better than random guess

In [9]:
A_err_rand = []
B_err_rand = []

for i in range(1000):
    A_err_rand.append(np.abs(A_true - \
                             np.random.uniform(size=A_true.shape)).mean())
    B_err_rand.append(np.abs(B_true - \
                             np.random.uniform(size=B_true.shape)).mean())

print("Random guess...")
print("Average abs error, transition matrix: {:.3f}" \
            .format(np.mean(A_err_rand)))
print("Average abs error, emission matrix: {:.3f}" \
            .format(np.mean(B_err_rand)))

Random guess...
Average abs error, transition matrix: 0.334
Average abs error, emission matrix: 0.354


Let's make the same estimations using Baum-Welch algorithm

In [10]:
%%capture
trainer = hmm.HiddenMarkovModelTrainer(states=states, symbols=symbols)
model_bw = trainer.train_unsupervised([list((obs[0],) for obs in seq)])

A_bw = get_probabilities_matrix(model_bw, kind="transition")
B_bw = get_probabilities_matrix(model_bw, kind="emission")

... not too far from random guess. Probably, this is because of small sequence length for this approach.

In [11]:
print("Unsupervised estimation...")
print("Average abs error, transition matrix: {:.3f}" \
          .format(np.abs(A_true - A_bw).mean()))
print("Average abs error, emission matrix: {:.3f}" \
          .format(np.abs(B_true - B_bw).mean()))

Unsupervised estimation...
Average abs error, transition matrix: 0.185
Average abs error, emission matrix: 0.270


And lastly, try unsupervised approach on top of supervised estimation

In [12]:
%%capture
trainer = hmm.HiddenMarkovModelTrainer(states=states, symbols=symbols)
model_cmb = trainer.train(labeled_sequences=[seq], 
                          unlabeled_sequences=[list((obs[0], "") for obs in seq)])

A_cmb = get_probabilities_matrix(model_cmb, kind="transition")
B_cmb = get_probabilities_matrix(model_cmb, kind="emission")

... worse than MLE algorithm, so let's settle on supervised (MLE) approach for further analysis.

In [13]:
print("Combined estimation...")
print("Average abs error, transition matrix: {:.3f}" \
          .format(np.abs(A_true - A_cmb).mean()))
print("Average abs error, emission matrix: {:.3f}" \
          .format(np.abs(B_true - B_cmb).mean()))

Combined estimation...
Average abs error, transition matrix: 0.107
Average abs error, emission matrix: 0.136


Now we can generate **test** sequence using **true** model ...

In [14]:
seq_test = model_true.random_sample(rng=np.random, length=365)
hidden_test_true = [obs[1] for obs in seq_test]

... and try to recover sequence of hidden states knowing only observed states (using our best model)

In [15]:
hidden_test_pred = model_mle.best_path([obs[0] for obs in seq_test])
print(classification_report(hidden_test_true, hidden_test_pred))

             precision    recall  f1-score   support

      angry       0.44      0.74      0.55        98
      happy       0.54      0.44      0.49       102
     normal       0.51      0.29      0.37       111
      tired       0.47      0.46      0.47        54

avg / total       0.49      0.48      0.47       365



Prediction quality is around $50\%$. Seems to be low, but let's try with random prediction:

In [16]:
hidden_test_rand = np.random.choice(a=states, size=365)
print(classification_report(hidden_test_true, hidden_test_rand))

             precision    recall  f1-score   support

      angry       0.33      0.33      0.33        98
      happy       0.26      0.23      0.24       102
     normal       0.29      0.23      0.26       111
      tired       0.15      0.24      0.18        54

avg / total       0.27      0.26      0.26       365



... and using model with **true** parameters:

In [17]:
hidden_test = model_true.best_path([obs[0] for obs in seq_test])
print(classification_report(hidden_test_true, hidden_test))

             precision    recall  f1-score   support

      angry       0.50      0.61      0.55        98
      happy       0.49      0.57      0.52       102
     normal       0.47      0.32      0.38       111
      tired       0.52      0.50      0.51        54

avg / total       0.49      0.49      0.48       365



We have obtained roughly the same quality with **true** parameters. <br>
It means that there is a lot of uncertainty in underlying data, our fitted model did its best, and the rest is randomness, which is unpredictable.